In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
train = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-training.csv')
test = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-test.csv')

In [0]:
train.head()

In [0]:
test.head()

In [0]:
train2 = train.drop(['Unnamed: 0','SeriousDlqin2yrs'],axis =1)
train2.head()
train2.isna().sum()

In [0]:
# train2 = train2.fillna(0) 
# 결측치 대신 0을 넣어줄수있다.

In [0]:
# 0 대신 평균이나 중위수를 넣어준다.

# train2['MonthlyIncome'] = train2['MonthlyIncome'].fillna(train2['MonthlyIncome'].mean())
# train2['NumberOfDependents'] = train2['NumberOfDependents'].fillna(train2['NumberOfDependents'].mean())

In [0]:
# monthly income 값을 보면 튀는 값들이 너무 많다! 그래서 평균값을 넣는것보다 0을 넣는것이 더 효율적일수 있다!
# -> 결측치를 알아서 채우는 모델링이 있다.

In [0]:
test2 = test.drop(['Unnamed: 0','SeriousDlqin2yrs'],1)
test2.head()

In [0]:
# test2 = test2.fillna(0)

In [0]:
# test2['MonthlyIncome'] = test2['MonthlyIncome'].fillna(test2['MonthlyIncome'].mean())
# test2['NumberOfDependents'] = test2['NumberOfDependents'].fillna(test2['NumberOfDependents'].mean())

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
from xgboost import XGBClassifier 
# 부스팅모델은 나무하나가 만들어지면서 앞에 나무를 참고하여 최적화하여(오답노트) 나무를 만든다.but,부작용(train set과적합이 되면 될수록 test set의 점수가 안나온다.)
# 그래서 xgboost 모델보다는 랜덤포레스트를 쓴다. 굉장히 섬세하고 민감하기떄문에 데이터를 잘파악하고 이해하고있을때문 쓰고 하이퍼 파라미터를 적재적소에 설정해준다.
# max_depth가 defualt가 3으로 되어있다. -> 질문을 굉장히 작게하므로 질문을 많이 하게해야한다.
# max_depth의 갯수를 봐가면서 설정해줘야함 -> 갯수를 낮췃는데도 점수가 같으면 우리 모델이 이미 과적합을 했다고 볼수있음

xgb = XGBClassifier(nthread=4,max_depth =6)
xgb.fit(train2,train['SeriousDlqin2yrs'])
prediction_xgb = xgb.predict_proba(test2)

In [0]:
from lightgbm import LGBMClassifier

# lgbm은 데이터의 복잡도를 설정할때 하이퍼 파라미터가 다르다.
# 학습할때 방식이 편식을 한다. 깊이 자체를 학습하는게 아니라 노드 기준으로한다(대용량 셋에 아주 유리함)
# num_leaves= :max_depth와 같음
# colsample_bytree= ((default =1))
# 1) ex 0.8 -> 나무를 가져올때 다가져오는것이아니라 80% 정도로 랜덤하게 뽑아감. ->test셋과 데이터가 많이다를떄.
# 2) 넣었을때 점수가 낮아졌더라도 없애버리지말고, 다른 하이퍼파라미터를 만져보고 점수비교해보고 colsample_bytree값을 넣어준다. 
# colsample_bytree를 넣었을때 점수가 올라가면 train set에서 중요한 것을 덜학습하고 안중요한것을 덜 학습한것이다.

lgbm = LGBMClassifier(colsample_bytree=0.8)
lgbm.fit(train2,train['SeriousDlqin2yrs'])
prediction_lgbm = lgbm.predict_proba(test2)

In [0]:
rf = RandomForestClassifier(n_estimators =100, n_jobs= -1,random_state=33,max_depth=8)

In [0]:
# 우리데이터셋의 복잡도 ,분포를 이해하고 파악하면 하이퍼파라미터를 결정하여 최적화된 점수 얻을 가능 있음.
# 복잡도를 판단하는 방법
# 1) 데이터가 많은지 적은지를 고려한다.
# 2) 칼럼의 갯수가 많으면 복잡해진다.
# 3) ** 범주형 변수가 있으면 그중에서 한변수가 카테고리가 아주많으면(계절1,2,3,4가아닌 어떤 지점)

# exception. y값에 유의미하지 컬럼이라면 학습할때 제외함.
#### 변수 중요도를 보고 판단한다. ### 시작전에 어떻게할지 그려봐야함

# 랜덤포레스트 자체가 굉장히 복잡하게 학습을함.
# 범주형 카테고리가 없기때문에 복잡하게 할필요없음 -> 하이퍼 파라미터를 바꿔줘야함
# ** max_depth 값의 기본값을 바꿔줘야함(기본값은 깊어질대로 깊어지도록 설정되어있음(max_depth=-1) 
# max_depth = 6 : 질문의 갯수를 2의 6제곱의 갯수로 질문을 한다.

In [0]:
# rf.fit(train2,train['SeriousDlqin2yrs'])

In [0]:
# prediction = rf.predict_proba(test2)

In [0]:
prediction

In [0]:
sub =pd.read_csv('/kaggle/input/GiveMeSomeCredit/sampleEntry.csv')

In [0]:
sub.head()

In [0]:
sub['Probability'] = prediction_xgb[:,1] * 0.3 + prediction_lgbm[:,1] * 0.7

In [0]:
sub.head()

In [0]:
sub.to_csv('second_class.csv', index =False)